In [ ]:
import sys

!{sys.executable} -m pip install ijson

In [ ]:
import datetime
import hpq
import ijson

start = datetime.datetime.fromtimestamp(1663281000, datetime.timezone.utc)
end = datetime.datetime.fromtimestamp(1663367400, datetime.timezone.utc)
connection = hpq.create_web_socket_client()
request = {
    "query": "ticks",
    "source": "impact_chi1_fod",
    "messages": ["TRD"],
    "product": "SB  FMV0022-SB  FMH0023",
    "time_zone": "UTC",
    "start_date": start.strftime("%Y-%m-%d"),
    "end_date": end.strftime("%Y-%m-%d"),
    "start_time": start.strftime("%H:%M:%S"),
    "end_time": end.strftime("%H:%M:%S"),
}
results = {"prices": [], "quantities": [], "timestamps": []}
for trade in ijson.items(connection.stream(request), "item"):
    results["prices"].append(trade["price"])
    results["quantities"].append(trade["quantity"])
    results["timestamps"].append(
        datetime.datetime.fromtimestamp(
            trade["exchange_timestamp"] / 1000000000, datetime.timezone.utc
        )
    )

In [ ]:
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.rcParams["figure.figsize"] = [10, 5]

fig, ax = plt.subplots()

fig.patch.set_facecolor((1, 1, 1))

ax.grid(True)

ax.set_title("SB  FMV0022-SB  FMH0023 trades, 2022-09-16")
ax.set_ylabel("Price")
ax.set_xlabel("Hour")

ax.set_ylim(0.25, 0.4)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%H"))

plt.plot(results["timestamps"], results["prices"])
plt.show()